# TFG: Título del TFG

## Hugo López Álvarez

In [ ]:
import math
import numpy    
import pandas   
import wandb
import torch    
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, log_loss, fbeta_score


## Clases

Definición de la clase DatasetTFG que se usará para entrenar al modelo

In [30]:
class DatasetTFG(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

Definición de la clase Modelo
- La capa1 transforma la dimensión de entrada a 64 neuronas
- La capa2 pasa de las 64 neuronas a 1 neurona

In [31]:
class Modelo(nn.Module):
    def __init__(self, input_dim, ventanaOculta):
        super().__init__()
        self.capa1 = nn.Linear(input_dim, ventanaOculta)
        self.capa2 =  nn.Linear(ventanaOculta, 1)
        self.sigprueba = nn.Sigmoid() #Se pasa el volor por una sigmoidea
        
    def forward(self,  X):
        X = self.capa1(X)
        X = self.capa2(X)
        #X = self.sigprueba(X)
        return X

# Funciones

In [32]:

def split_ip_column(df, ip_column_name):
    
    # Divide la IP en cuatro partes
    ip_parts = df[ip_column_name].str.split('.', expand=True)
    
    # Crea nombres de columnas basados en el nombre original
    new_columns = {
        0: f"{ip_column_name}_part1",
        1: f"{ip_column_name}_part2", 
        2: f"{ip_column_name}_part3",
        3: f"{ip_column_name}_part4"
    }
    
    # Se elimina la columna de ip_column_name
    df = df.drop(columns=[ip_column_name]) 
    
    # Añade las nuevas columnas al DataFrame
    for part, col_name in new_columns.items():
        df[col_name] = pandas.to_numeric(ip_parts[part])  # Convierte a numérico
    
    return df

## Cargar datos

In [33]:
fileData = pandas.read_csv('../Datasets/modUQ.csv')

### Comprobación de la obtención correcta del csv

In [34]:
fileData.head()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,...,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label,Attack
0,1424242193040,1424242193043,59.166.0.2,4894,149.171.126.3,53,17,5.0,146,2,...,0,0,0,0,0,0,0,0,0,Benign
1,1424242192744,1424242193079,59.166.0.4,52671,149.171.126.6,31992,6,11.0,4704,28,...,0,91,12,19,0,90,12,19,0,Benign
2,1424242190649,1424242193109,59.166.0.0,47290,149.171.126.9,6881,6,37.0,13662,238,...,0,1843,10,119,0,1843,5,88,0,Benign
3,1424242193145,1424242193146,59.166.0.8,43310,149.171.126.7,53,17,5.0,146,2,...,0,0,0,0,0,0,0,0,0,Benign
4,1424242193239,1424242193241,59.166.0.1,45870,149.171.126.1,53,17,5.0,130,2,...,0,0,0,0,0,0,0,0,0,Benign


### Se convierten las columnas no numéricas para poder utilizarlas con pytorch

In [35]:
fileData = split_ip_column(fileData, 'IPV4_SRC_ADDR')
fileData = split_ip_column(fileData, 'IPV4_DST_ADDR')
fileData['Attack'] = LabelEncoder().fit_transform(fileData['Attack'])

### Se comprueba que los datos se han transformado correctamente

In [36]:
print(fileData.dtypes)

FLOW_START_MILLISECONDS    int64
FLOW_END_MILLISECONDS      int64
L4_SRC_PORT                int64
L4_DST_PORT                int64
PROTOCOL                   int64
                           ...  
IPV4_SRC_ADDR_part4        int64
IPV4_DST_ADDR_part1        int64
IPV4_DST_ADDR_part2        int64
IPV4_DST_ADDR_part3        int64
IPV4_DST_ADDR_part4        int64
Length: 61, dtype: object


## Se eliminan los datos con valores infinitos

In [37]:
print("¿Existen valores infinitos en X?: ", numpy.isinf(fileData.values).any())
fileData = fileData.replace([numpy.inf, -numpy.inf], numpy.nan).dropna()
print("¿Siguen existiendo valores infinitos en X?: ", numpy.isinf(fileData.values).any())

¿Existen valores infinitos en X?:  True
¿Siguen existiendo valores infinitos en X?:  False


### Se separan las características (X) de la etiqueta (Y)

In [38]:
X = fileData.drop(columns=['Label', 'Attack']).values
Y = fileData['Label'].values

## Se separan los datos del entrenamiento de los datos de prueba
El entrenamiento tendrá el 80% de los datos

La prueba tendrá el 20% de los datos

In [39]:
X_entrena, X_prueba, Y_entrana, Y_prueba = train_test_split(
    X, Y, test_size=0.2, random_state=42,  stratify=Y
)

## Se normalizan los datos

In [40]:
escalador = MinMaxScaler(feature_range=(0,1))
X_entrena_normalizado = escalador.fit_transform(X_entrena)

### Se convierten los datos a tensores de Pytorch

In [41]:
X_entrena_tensor = torch.tensor(X_entrena_normalizado, dtype=torch.float32)
Y_entrena_tensor = torch.tensor(Y_entrana, dtype=torch.long)

## Creación del Dataset personalizado

In [42]:
dataset_entrena = DatasetTFG(X_entrena_tensor, Y_entrena_tensor)

## Se configura pérdida y optimizador

In [ ]:
#perdida = nn.BCELoss() if len(Y_entrena_tensor.unique()) > 2 else nn.CrossEntropyLoss()
pos_weight = torch.tensor(19.0)
perdida = nn.BCEWithLogitsLoss(pos_weight=pos_weight) #95/5

## Se preparan los datos de prueba

In [ ]:
X_prueba_normalizado = escalador.fit_transform(X_prueba)

X_prueba_tensor = torch.tensor(X_prueba_normalizado, dtype=torch.float32)
Y_prueba_tensor = torch.tensor(Y_prueba, dtype=torch.long)

dataset_prueba = DatasetTFG(X_prueba_tensor, Y_prueba_tensor)


## Definición de hiperparámetros

In [ ]:
batch_size=[2000, 10000, 15000, 20000]
learning_rate=[0.01, 0.001, 0.0001]
epochs=[10, 20, 30]
hidden_factor=[math.ceil(X_entrena_tensor.size(1)/2), X_entrena_tensor.size(1), X_entrena_tensor.size(1)*2] # la mitad de las columnas, el número de columnas y el doble

## Bucle de entrenamiento o épocas

In [ ]:
for bs in batch_size:
    for lr in learning_rate:
        for hs in hidden_factor:
            for e in epochs:
                # Configuración del experimento en wandb
                nombreExperimento = f'TFG_BIN_bs({bs})_lr({lr})_hs({hs})_e({e})'
                wandb.init(
                    project="TFG_BIN",
                    name=nombreExperimento,
                    config={
                        "batch_size": bs,
                        "learning_rate": lr,
                        "hidden_size": hs,
                        "epochs": e
                    }
                )

                # DataLoaders
                train_loader = DataLoader(dataset_entrena, batch_size=bs, shuffle=True) #se cogen los datos de manera aleatoria (shuffle)
                val_loader = DataLoader(dataset_prueba, batch_size=bs)

                # Modelo, optimizador y función de pérdida
                modelo = Modelo(input_dim= X_entrena_tensor, ventanaOculta=hs)
                optimizador = optim.AdamW(modelo.parameters(), lr=lr)

                # Entrenamiento
                for epoch in range(e):
                    modelo.train()
                    for batch_X, batch_Y in train_loader:
                        optimizador.zero_grad()
                        salidas = modelo(batch_X)
                        loss = perdida(salidas, batch_Y.unsqueeze(1))
                        loss.backward()
                        optimizador.step()

                # Evaluación (métricas finales)
                modelo.eval()
                val_preds = []
                val_probs = []
                val_targets = []
                val_loss = 0.0

                with torch.no_grad():
                    for batch_X_val, batch_Y_val in val_loader:
                        salidas_val = modelo(batch_X_val)
                        val_loss += perdida(salidas_val, batch_Y_val.unsqueeze(1)).item()
                        probs = torch.sigmoid(salidas_val)
                        preds = (probs > 0.5).int()
                        val_probs.extend(probs.cpu().numpy())
                        val_preds.extend(preds.cpu().numpy())
                        val_targets.extend(batch_Y_val.cpu().numpy())

                # Cálculo de métricas
                val_loss /= len(val_loader)
                accuracy = accuracy_score(val_targets, val_preds)
                precision = precision_score(val_targets, val_preds, zero_division=0)
                recall = recall_score(val_targets, val_preds, zero_division=0)
                f1 = f1_score(val_targets, val_preds)
                f2 = fbeta_score(val_targets, val_preds, beta=2)
                roc_auc = roc_auc_score(val_targets, val_probs)
                tn, fp, fn, tp = confusion_matrix(val_targets, val_preds).ravel()
                specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

                # Log en wandb
                wandb.log({
                    "loss": val_loss,
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1": f1,
                    "f2": f2,
                    "roc_auc": roc_auc,
                    "specificity": specificity,
                    "true_positives": tp,
                    "false_positives": fp,
                    "true_negatives": tn,
                    "false_negatives": fn
                })

                print(f"\nExperimento: {nombreExperimento}")
                print(f"  Loss: {val_loss:.4f} | Accuracy: {accuracy:.4f} | F1: {f1:.4f} | AUC: {roc_auc:.4f}")

                wandb.finish()

## Se guarda el modelo en un fichero

In [ ]:
torch.save(modelo.state_dict(), 'modeloSinLabelNiAttackNiIPSRC.pth')

### BCEWithLogitsLoss

Época: 1, Pérdida: 0.020396
Época: 2, Pérdida: 0.009877
Época: 3, Pérdida: 0.004355
Época: 4, Pérdida: 0.00114
Época: 5, Pérdida: 0.002756
Época: 6, Pérdida: 0.001659
Época: 7, Pérdida: 0.004422
Época: 8, Pérdida: 0.000846
Época: 9, Pérdida: 0.000706
Época: 10, Pérdida: 0.000514

Pérdida durante la prueba: 0.0075, Exactitud:  95.90%
